In [1]:
import sqlite3
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from urllib.request import urlopen
from json import load
import os
from glob import glob
import PyPDF2
sns.set_style('white')
sns.set_context('notebook')

# 1. Imports

In [2]:
pdf_file = open('../recruitment/payments/Amazon.com - Order 112-7833647-0376253.pdf', 'rb')
pdf_reader = PyPDF2.PdfFileReader(pdf_file)
page = pdf_reader.getPage(0)
ptest = page.extractText()
xx= ptest.split('E-mail gift card to: ')[1:]
[x.split("\n")[0] for x in xx]

['cocoroddy94@gmail.com',
 'ngood2600@gmail.com',
 'stviloveyou@gmail.com',
 'mattewgretchen@gmail.com',
 'aiyanakaren657754@gmail.com',
 'soonwow041@gmail.com']

# 2. Subjects who have been paid

In [2]:
paid_subjs = []
receits = glob('../recruitment/payments/Amazon*')
for filepath in receits:
    pdf_reader = PyPDF2.PdfFileReader(open(filepath, 'rb'))
    for page in range(pdf_reader.numPages):
        pg = pdf_reader.getPage(page).extractText()
        xx = pg.split('E-mail gift card to: ')[1:]
        paid_subjs.append([x.split("\n")[0] for x in xx])

paid_subs = [item.lower() for sublist in paid_subjs for item in sublist]
#paid_subs

In [3]:
payments = pd.DataFrame({'email': paid_subs, "date_paid": datetime.datetime(2022, 9, 12)})
#payments.to_csv('../recruitment/payments/paid_subs.csv', sep=',', header=True)
payments

,email,date_paid
0,victoria.gresbach@gmail.com,2022-09-12
1,lunaoliver031@gmail.com,2022-09-12
2,austinhill113@gmail.com,2022-09-12
3,olamidegsax@gmail.com,2022-09-12
4,clarkdaniel781@gmail.com,2022-09-12
...,...,...
464,mleshc15@gmail.com,2022-09-12
465,kenneththompkins860@gmail.com,2022-09-12
466,scottwiliams1894@gmail.com,2022-09-12
467,paigesummer44@yahoo.com,2022-09-12


In [6]:
topay = pd.read_csv('./Data/subs2pay_10_10_2022.csv', index_col=0)
topay = [x[0] for x in topay.values.tolist()]
topay

['drewhartado@gmail.com',
 'ethanbrandon.j@gmail.com',
 'sellasmallen37@gmail.com',
 'josehastings8@gmail.com',
 'steffi.lietzke@gmail.com',
 'lonniekacie125@yahoo.com',
 'lamperdominic@gmail.com',
 'michaeltaylor.v63@gmail.com',
 'stevegutierrez305@gmail.com',
 'beckyybanks@gmail.com',
 'feroztino856@gmail.com',
 'jamesjensen642@gmail.com',
 'canndyvelyn@gmail.com',
 'botspeterson@gmail.com',
 'owensnancy600@gmail.com',
 'mcharlescalderon@gmail.com',
 'marystafford554@gmail.com',
 'viviansmelley680@gmail.com',
 'cathylee760@gmail.com',
 'jeffreymonroe074@gmail.com',
 'webbo9777@gmail.com',
 'carissabhalliday@gmail.com',
 'darnellhayes465@gmail.com',
 'stella.sage56@gmail.com',
 'terryjohnjtt767@gmail.com',
 'ninaschmidhuan@gmail.com',
 'debramcuire@gmail.com',
 'catherinewilkerrson@gmail.com',
 'johnyyavant@gmail.com',
 'andersongillian930@gmail.com',
 'daniebrown.d@gmail.com',
 'shamnikeller@gmail.com',
 'ryanronald251@gmail.com',
 'jy2926@nyu.edu',
 'musicmajor2016@gmail.com',
 'c.g

In [7]:

paythem = []
for subject in topay:
    if len(payments[payments.email.str.contains(subject)]) < 1:
        paythem.append(subject)



In [7]:
# add some subjects manually
#[paythem.append(x) for x in ['acoustajavier@gmail.com', 'idencloak@gmail.com', 'loreyclaxtone@gmail.com']]
newpayments = pd.DataFrame({'email': paythem, 'date_paid': datetime.datetime(2022, 10, 3)})

In [12]:
# wait to pay mailfence people.


new = newpayments[['mailfence' not in x for x in  newpayments.email]]
foreign = ['loreneriley591@gmail.com','ebarlow6403@outlook.com','lmmaier424@msn.com','mattbradford29@gmail.com','nerysgross@yahoo.co.uk', 'kateellis00@gmail.com']

new[new.email.isin(foreign)]

,email,date_paid
29,loreneriley591@gmail.com,2022-10-03
44,mattbradford29@gmail.com,2022-10-03
50,nerysgross@yahoo.co.uk,2022-10-03
57,lmmaier424@msn.com,2022-10-03
66,ebarlow6403@outlook.com,2022-10-03
68,kateellis00@gmail.com,2022-10-03


In [15]:
pd.concat([payments, newpayments], axis=0).to_csv('../recruitment/payments/paid_subs.csv', sep=',', header=True)

# 3. Subjects who need to be paid

In [26]:
conn = sqlite3.connect('./Data/knwlg_blf.db')
subjs = pd.read_sql("SELECT * FROM subjects WHERE participation_date > 8/24/2022 "
                   "AND block1_complete=TRUE "
                   "AND block2_complete=TRUE "
                   "AND block3_complete=TRUE "
                   "AND completion_code LIKE '%XXX%'", con=conn)

# 4. Updating record of paid subjects

In [5]:
payments = pd.read_csv('../recruitment/payments/paid_subs.csv', index_col=0, header=0)


In [10]:
payments.loc[payments.date_paid.isnull()]

,email,date_paid


In [13]:
sum(payments.email.str.find(subject) != -1)
len(payments[payments.email.str.contains(subject)])

1